# Load packages

In [1]:
import os
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
from scripts.evalModel import evalLogisticModels, getPredDist

# Data

## Load cell lines

In [3]:
trainRNA = pd.read_csv('../../data/processed/RNA_train_cancergenes.csv', index_col=0)
trainCellLines = list(trainRNA.index)

testRNA = pd.read_csv('../../data/processed/RNA_test_cancergenes.csv', index_col=0)
testCellLines = list(testRNA.index)

newRNA = pd.read_csv('../../data/processed/RNA_newcancer_cancergenes.csv', index_col=0)
newCellLines = list(newRNA.index)

## CDR

In [4]:
cdr = pd.read_csv('../../data/processed/drugCellLinePairsData.csv', index_col='DepMap_ID')
trainCDR = cdr.loc[trainCellLines, :].reset_index()
testCDR = cdr.loc[testCellLines, :].reset_index()
newCDR = cdr.loc[newCellLines, :].reset_index()

In [5]:
testTemp = testCDR.loc[:, ['DepMap_ID', 'cancer_type', 'name', 'effective']].rename(columns={'DepMap_ID':'cell_line',
                                                                                             'name': 'drug',
                                                                                             'effective': 'true'})

newTemp = newCDR.loc[:, ['DepMap_ID', 'cancer_type', 'name', 'effective']].rename(columns={'DepMap_ID':'cell_line',
                                                                                          'name': 'drug',
                                                                                          'effective': 'true'})

# Load drugs

In [6]:
drugs = pd.read_csv('../../data/processed/drug_fingerprints.csv', index_col=0)

drugs.head()

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
cytarabine,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
epinastine,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
floxuridine,0,0,0,0,1,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,0
valrubicin,1,0,0,0,0,1,0,0,1,0,...,0,1,0,1,1,1,0,1,0,0
adapalene,1,1,1,0,0,1,0,0,0,0,...,0,0,1,1,1,1,0,0,0,0


In [7]:
trainDrugs = drugs.loc[list(trainCDR.name.values), :].to_numpy()
testDrugs = drugs.loc[list(testCDR.name.values), :].to_numpy()
newDrugs = drugs.loc[list(newCDR.name.values), :].to_numpy()

trainRNA = trainRNA.loc[list(trainCDR.DepMap_ID.values), :].to_numpy()
testRNA = testRNA.loc[list(testCDR.DepMap_ID.values), :].to_numpy()
newRNA = newRNA.loc[list(newCDR.DepMap_ID.values), :].to_numpy()

In [8]:
trainData = [trainDrugs, trainRNA]
trainEff = trainCDR.effective.to_numpy()
testData = [testDrugs, testRNA]
newData = [newDrugs, newRNA]

In [9]:
del cdr, drugs, trainDrugs, testDrugs, newDrugs, trainRNA, testRNA, newRNA

# Fusion performance

In [10]:
fusionPath = '/fs/scratch/PCON0041/PatrickLawrence/cancer-drug-response/fewShot/fusion/rawDrug_rawRNA/models'
bestModel = 'FusionFewShotRawDrugRawCell_NL128_64_32_DO0-1_AFrelu_LR0-001_DR0-99_DS1024_BYrna'
fusionPath = os.path.join(fusionPath, bestModel)

In [12]:
evalLog = evalLogisticModels(trainData.copy(), trainEff, testData.copy(), newData.copy(), alt='xgb',
                            fusionPath=fusionPath, drugPath=None, rnaPath=None)

In [13]:
testDF, newDF, testWrong, newWrong, testCounts, newCounts = evalLog.evaluate(testDF=testTemp.copy(), 
                                                                            newDF=newTemp.copy(),
                                                                            thresh=0.1)


Average Cell Line precision @ k on test set
	Precision@1: 0.4314
	Precision@2: 0.4706
	Precision@3: 0.4837
	Precision@4: 0.4951
	Precision@5: 0.498
	Precision@10: 0.5615

Average Cell Line precision @ k on newcancer set
	Precision@1: 0.4923
	Precision@2: 0.4462
	Precision@3: 0.4821
	Precision@4: 0.5
	Precision@5: 0.5169
	Precision@10: 0.6216


Test set:
No true effective drugs identified in top 3 for ACH-000222 (top drug: XL-647)
No true effective drugs identified in top 3 for ACH-000329 (top drug: tanespimycin)
No true effective drugs identified in top 3 for ACH-000493 (top drug: tanespimycin)
No true effective drugs identified in top 3 for ACH-000510 (top drug: rutin)
No true effective drugs identified in top 3 for ACH-000563 (top drug: mitoxantrone)
No true effective drugs identified in top 3 for ACH-000650 (top drug: valrubicin)
No true effective drugs identified in top 3 for ACH-000663 (top drug: XL-647)
No true effective drugs identified in top 3 for ACH-000665 (top drug: ryuvidi

In [14]:
testDist = getPredDist(testDF)

Avg varaince of predictions for each drug: 0.001500000013038516


In [15]:
newDist = getPredDist(newDF)

Avg varaince of predictions for each drug: 0.0017000000225380063


In [16]:
testCounts.head()

,1,2,3,total
tanespimycin,7,4,1,12
XL-647,3,1,2,6
OTS167,3,1,2,6
mitoxantrone,2,5,6,13
dolastatin-10,2,2,2,6


In [17]:
# poor cell line
testDF[testDF.cell_line == 'ACH-000161'].head(10)

,cell_line,cancer_type,drug,true,pred
6235,ACH-000161,Lung Cancer,cabazitaxel,0,0.561632
6289,ACH-000161,Lung Cancer,AZD8330,0,0.561632
6259,ACH-000161,Lung Cancer,sangivamycin,1,0.561632
6262,ACH-000161,Lung Cancer,JNJ-26481585,0,0.561632
6261,ACH-000161,Lung Cancer,epothilone-d,1,0.561632
6278,ACH-000161,Lung Cancer,brilliant-green,1,0.561632
6234,ACH-000161,Lung Cancer,docetaxel,0,0.561073
6284,ACH-000161,Lung Cancer,gemcitabine,1,0.551332
6310,ACH-000161,Lung Cancer,rubitecan,0,0.551332
6274,ACH-000161,Lung Cancer,YM-155,1,0.551332


In [18]:
# poor cell line
testDF[testDF.cell_line == 'ACH-000899'].head(10)

,cell_line,cancer_type,drug,true,pred
4538,ACH-000899,Skin Cancer,tanespimycin,0,0.561632
4505,ACH-000899,Skin Cancer,mitoxantrone,0,0.558912
4527,ACH-000899,Skin Cancer,sirolimus,0,0.558912
4545,ACH-000899,Skin Cancer,10-hydroxycamptothecin,1,0.517807
4496,ACH-000899,Skin Cancer,genz-644282,0,0.517807
4528,ACH-000899,Skin Cancer,alvespimycin,1,0.468076
4515,ACH-000899,Skin Cancer,JNJ-26481585,0,0.466994
4541,ACH-000899,Skin Cancer,OTS167,0,0.466994
4521,ACH-000899,Skin Cancer,epothilone-b,0,0.449094
4534,ACH-000899,Skin Cancer,gemcitabine,0,0.449094


## Cancer precision

In [19]:
cancerTest, cancerNew = evalLog.getCancerPerformance(testDF.copy(), newDF.copy())

In [20]:
print(round(cancerTest.mean(), 4))
cancerTest

p1    0.4149
p2    0.4697
p3    0.5018
p4    0.5077
p5    0.5116
dtype: float64


,p1,p2,p3,p4,p5
Bladder Cancer,1.000000,0.833333,0.888889,0.833333,0.733333
Brain Cancer,0.750000,0.750000,0.583333,0.562500,0.550000
Head and Neck Cancer,0.666667,0.666667,0.666667,0.666667,0.733333
Skin Cancer,0.600000,0.300000,0.333333,0.400000,0.360000
Ovarian Cancer,0.500000,0.500000,0.500000,0.500000,0.550000
Liver Cancer,0.500000,0.250000,0.333333,0.375000,0.400000
Lung Cancer,0.461538,0.461538,0.410256,0.442308,0.446154
Colon/Colorectal Cancer,0.250000,0.500000,0.583333,0.562500,0.600000
Pancreatic Cancer,0.250000,0.375000,0.500000,0.500000,0.500000
Endometrial/Uterine Cancer,0.000000,0.500000,0.555556,0.583333,0.666667


In [21]:
print(round(cancerNew.mean(), 4))
cancerNew

p1    0.5402
p2    0.4959
p3    0.5247
p4    0.5216
p5    0.5417
dtype: float64


,p1,p2,p3,p4,p5
Prostate Cancer,1.000000,0.750000,0.666667,0.625000,0.600000
Gallbladder Cancer,1.000000,0.500000,0.666667,0.500000,0.600000
Gastric Cancer,0.785714,0.464286,0.523810,0.517857,0.514286
Bone Cancer,0.666667,0.611111,0.666667,0.638889,0.666667
Bile Duct Cancer,0.600000,0.600000,0.533333,0.450000,0.440000
Neuroblastoma,0.333333,0.666667,0.555556,0.583333,0.533333
Sarcoma,0.333333,0.333333,0.333333,0.458333,0.566667
Kidney Cancer,0.307692,0.346154,0.384615,0.442308,0.446154
Rhabdoid,0.250000,0.500000,0.666667,0.687500,0.700000
Thyroid Cancer,0.125000,0.187500,0.250000,0.312500,0.350000
